In [1]:
from torch.nn import Sequential, Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d, LeakyReLU, GELU, SELU, Mish
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math

In [2]:
def get_data(param, type):
    if(type.lower() == 'train'):
        transform = transforms.Compose([
            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomRotation(degrees=12),
            transforms.ColorJitter(),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])

        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    
    else:
        transform = transforms.Compose([
            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        test_dataset = datasets.ImageFolder(root=param['test_data_dir'], transform=transform)
        test_dataloader = DataLoader(test_dataset, batch_size=param['batch_size'])
        return test_dataloader

## Transfer Learning using VGG16 model

In [41]:
def train():

    # wandb.init()
    # param = wandb.config
    # wandb.run.name = f'vgg16_drpOut_{param.dropout}_batchSz_{param.batch_size}_neurons_{param.dense_neurons}_epochs_{param.epochs}'

    param = {
        "dropout": 0.3,
        "batch_size": 32,
        "epochs": 5,
        "dense_neurons": 128,
        "train_data_dir": "./data/train",
        "test_data_dir": "./data/val"
    }


    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = models.vgg16(pretrained=True)

    # print(model.classifier[3].in_features)
    # print(model.classifier)
    # print(model)
    new_classifier = Sequential(
        Linear(in_features=25088, out_features=4096, bias=True),
        ReLU(inplace=True),
        Dropout(p=0.5, inplace=False),
        Linear(in_features=4096, out_features=4096, bias=True),
        ReLU(inplace=True),
        Dropout(p=0.5, inplace=False),
        Linear(in_features=4096, out_features=10, bias=True)
    )
    model.classifier = new_classifier
    for x in model.parameters():
        x.requires_grad = False
    model.classifier[5].requires_grad = True
    model.classifier[6].requires_grad = True
    # print(model)
    for x in model.parameters():
        print(x.requires_grad)
    # model.classifier[6] = Sequential(
    #     Linear(in_features=4096, out_features=param['dense_neurons']), 
    #     ReLU(), 
    #     Dropout(param['dropout']),
    #     Linear(in_features=param['dense_neurons'], out_features=10),                   
    #     LogSoftmax(dim=1)
    # )
    # model.classifier[6] = L
    
    
    
    
    # inear(in_features=model.classifier[3].in_features, out_features=param['dense_neurons'])
    # model.classifier.add(ReLU())
    # model.classifier.add(Dropout(param['dropout']))
    # print(model.classifier)

    '''
    total_params = sum(p.numel() for p in model.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} training parameters.')

    model = model.to(device)
    optimizer = Adam(model.parameters())
    loss_function = NLLLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')

    for epo in param['epochs']:
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = model(image)
            loss = loss_function(prediction, label)
            ## no optimize.zero_grad() ...
            loss.backward()
            optimizer.step()

            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1
            print(train_counter)

        
        for (image, label) in validation_data_loader:
            pred = model(image)
            loss = loss_function(pred, label)
            totalValLoss += loss
            valCorrect += (pred.argmax(1) == label).type(float).sum().item()
            validation_counter += 1

        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)
        print(f"Epoch --> {epo}")
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        
        # lg={
        #     'epoch': epo+1,
        #     'tr_accuracy': tr_acc,
        #     'val_accuracy': val_acc,
        #     'tr_loss': tr_ls,
        #     'val_loss': val_ls
        # }
        # wandb.log(lg)

    # torch.save(model, checkpoint_path)

    '''    
    

In [42]:
train()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [20]:
sweep_config = {
  "method": "grid",  # Use grid search for this example
  "name": "PartB VGG16 Sweep",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "dropout":{"values": [0.3]},  # Dropout rates
    "batch_size":{"values": [32]},
    "epochs":{"values": [5]},
    "dense_neurons":{"values": [128]},
    "train_data_dir":{"values": ["./data/train"]},
    "test_data_dir":{"values": ["./data/val"]}
  }
}

In [21]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment2")

Create sweep with ID: t1cg6a4y
Sweep URL: https://wandb.ai/cs23m070/cs6910_assignment2/sweeps/t1cg6a4y


In [22]:
wandb.agent(sweep_id, function=train, count=1)
wandb.finish()

wandb: Agent Starting Run: euvrg5lb with config:
wandb: 	batch_size: 32
wandb: 	dense_neurons: 128
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	test_data_dir: ./data/val
wandb: 	train_data_dir: ./data/train
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/Users/tirth/anaconda3/envs/dl/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tirth/anaconda3/envs/dl/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Run euvrg5lb errored:
Traceback (most recent call last):
  File "/Users/tirth/anaconda3/envs/dl/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/var/folders/6f/x1kv14zj6zqb4mjy03x74_m80000gn/T/ipykernel_1210/2580241278.py", line 13, in train
    Linear(in_features=4096, out_features=param['dense_neurons']),
IndexError: too many indices for tensor of dimension 1

wandb: ERROR Run euvrg5lb errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/Users/tirth/anaconda3/envs/dl/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/var/folders/6f/x1kv14zj6zqb4mjy03x74_m80000gn/T/ipykernel_1210/2580241278.py", line 13, in train
wandb: ERROR     Linear(in_features=4096, out_features=param['dense_neurons']),
wandb: ERROR IndexError: too many indices for tensor of dimension 1
wandb: ERROR 
